In [1]:
import pandas as pd
import psycopg2 as pg
import matplotlib.pyplot as mt
from sqlalchemy import create_engine

In [2]:
engine = create_engine('postgresql://localhost/john')

In [3]:
conn = engine.connect()

In [4]:
select = ("select dpd.issuer_code, dpd.transaction_date,dpd.open_price,dpd.high_price,dpd.low_price,dpd.close_price, dpd.daily_volume, row_number() over(partition by (dpd.issuer_code) order by (dpd.transaction_date)) as day_count from asx_database.daily_price_data dpd left join asx_database.listed_companies lc on lc.issuer_code = dpd.issuer_code where gics_industry_group = 'Software & Services' and dpd.transaction_date > '2018-01-01';")

In [5]:
daily_data_raw = pd.read_sql(select, conn)

In [6]:
conn.close()

In [7]:
daily_data_raw

,issuer_code,transaction_date,open_price,high_price,low_price,close_price,daily_volume,day_count
0,3DP,2018-01-02,0.145,0.145,0.140,0.140,840117,1
1,3DP,2018-01-03,0.140,0.160,0.140,0.160,2250569,2
2,3DP,2018-01-04,0.160,0.160,0.145,0.155,1669546,3
3,3DP,2018-01-05,0.155,0.155,0.145,0.145,1097063,4
4,3DP,2018-01-08,0.150,0.150,0.135,0.135,998233,5
...,...,...,...,...,...,...,...,...
97509,ZYB,2020-08-25,0.014,0.014,0.014,0.014,0,671
97510,ZYB,2020-08-26,0.014,0.014,0.014,0.014,0,672
97511,ZYB,2020-08-27,0.014,0.014,0.014,0.014,0,673
97512,ZYB,2020-08-28,0.014,0.014,0.014,0.014,0,674


In [ ]:
daily_data_raw

# Get distinct issuer codes

In [8]:
issuers = list(daily_data_raw['issuer_code'].drop_duplicates())

In [9]:
type(issuers)

list

In [ ]:
ax = mt.gca()
mt.figure(figsize=(10,10))
for issuer in issuers:
    issuer_data = daily_data_raw[daily_data_raw.issuer_code == issuer]
    issuer_data.plot(kind='line', x='day_count', y='close_price',legend='issuer_code', ax=ax)

In [27]:
data = daily_data_raw[["issuer_code", "close_price"]]

In [30]:
data = data.groupby('issuer_code')

In [ ]:
data.plot(x='issuer_code', kind='box')